# StyleGan2-ADA

official paper: https://arxiv.org/abs/2006.06676

official repo: https://github.com/NVlabs/stylegan2-ada

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import PIL
from PIL import Image
from pathlib import Path

In [ ]:
#In order to train the model this version of numpy must be installed
%pip install numpy==1.19.5

In [ ]:
#In order to train the model this version of TensorFlow must be installed
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Thu Feb 24 12:58:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

## Preparing  dataset

In [ ]:
# Defining an image size and image channel
# We are going to resize all our images to 1024x1024 size and since our images are colored images
# We are setting our image channels to 3 (RGB)

IMAGE_SIZE = 1024
IMAGE_CHANNELS = 3
IMAGE_DIR = '/content/drive/MyDrive/tesi2/images'

# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR
output_dataset_path = '/content/drive/MyDrive/tesi2/resized_jap_1024'

# Iterating over the images inside the directories and resizing them using
# Pillow's resize method.
print('resizing...')

cnt = 0

for artist in os.listdir(images_path):
  
  artist_path = os.path.join(images_path,artist)
  
  for file in os.listdir(artist_path):
  
    path = os.path.join(artist_path,file)
  
    im = Image.open(path)
  
    image = im.convert('RGB')
  
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

    filename = os.path.join(output_dataset_path, f"resized-{cnt}.png")
  
    image.save(filename)

    cnt +=1

print("All done! :) A total of: ", len(os.listdir(output_dataset_path)), "images has been resized to", IMAGE_SIZE, "x",IMAGE_SIZE )

resizing...


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (96709923 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


All done! :) A total of:  3562 images has been resized to 1024 x 1024


**NB**: at this point some other images were manually deleted as they weren't fit for the purpose.

### Images --> TFR

StyleGAN2-ADA takes as inpunt images in TFR format, for this reason the 1024x1024 images have to be transformed

In [ ]:
#Creating a repo where we will store the tfr
%mkdir 'path/to/tfr_repo'

In [ ]:
dataset_path = 'path/to/1024x1024images'
tfr_path = 'path/to/tfr_repo'

!python "stylegan2_repo_path/dataset_tool.py" create_from_images {tfr_path} {dataset_path}

Loading images from "/content/drive/MyDrive/HED/out"
Creating dataset "/content/drive/MyDrive/StyleGAN2-ADA/tfr_sketch"
/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 5000 images.


# Training

In [ ]:
#Be sure to have tf 1.x version installed!!
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Sun Feb 27 16:44:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

In [ ]:
!python "stylegan2_repo_path/train.py" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
!python "stylegan2_repo_path/train.py" --outdir="path/to/output_repo" --data="'path/to/tfr_repo'" \
    --snap=2 --augpipe='bgc'\
    --mirror=True --metrics=None --mirrory=False

"""
TRANSFER LEARNING: use the --resume='' option with the path to a pretrained net saved in a pkl file

ex:

!python "stylegan2_repo_path/train.py" --outdir="path/to/output_repo" --data="'path/to/tfr_repo'" \
    --snap=2 --augpipe='bgc' --resume='path/to/pretrained_models_repo/pretrained_net.pkl'\
    --mirror=True --metrics=None --mirrory=False --freezed=13

"""

Traceback (most recent call last):
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 645, in <module>
    main()
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 637, in main
    run_training(**vars(args))
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/train.py", line 522, in run_training
    training_loop.training_loop(**training_options)
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/training/training_loop.py", line 255, in training_loop
    tflib.run([D_train_op, Gs_update_op, Gs_epochs_op], {Gs_beta_in: Gs_beta, Gs_epochs: epochs})
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dnnlib/tflib/tfutil.py", line 33, in run
    return tf.get_default_session().run(*args, **kwargs)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1180, in _run
    feed

# Generate images
NB: If you generate images and then want to train again the model, be sure to install numpy 1.19 again as the next install will update it to 1.21.5

In [ ]:
%pip install opensimplex

In [ ]:
!python "stylegan2_repo_path/generate.py" generate-images --help

usage: generate.py generate-images [-h] --network NETWORK_PKL --seeds SEEDS
                                   [--trunc TRUNCATION_PSI]
                                   [--class CLASS_IDX] [--create-grid]
                                   [--outdir DIR] [--save_vector] [--fixnoise]
                                   [--jpg_quality JPG_QUALITY]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --create-grid         Add flag to save the generated images in a grid
  --outdir DIR          Root directory for run results (default: out)
  --save_vector         also save vector in .npy format
  --fixnoise            generate images using fixed noise (more accurate for
                        interpolations)
  

In [ ]:
from numpy import random
seed_init = random.randint(1)
nbr_images = 2

#insert here the path to the pretrained model you want to generate images with
generation_from = 'path/to/pretrained_models_repo/pretrained_net.pkl'

!python "stylegan2_repo_path/generate.py" generate-images \
    --outdir="path/to/output_repo/generated_repo" --trunc=0.5 \
    --seeds={seed_init}-{seed_init+nbr_images-1} \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training_imp_scratch/00009-tfr_imp-mirror-auto1-bgc-resumecustom/network-snapshot-000040.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 0 (0/2) ...
Generating image for seed 1 (1/2) ...


## Latent space walk

In [ ]:
%pip install opensimplex

In [ ]:
from numpy import random
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_path = 'path/to/output_repo/latent_walk'

#insert here the path to the pretrained model you want to explore
explored_network = 'path/to/pretrained_models_repo/pretrained_net.pkl'

seeds = [random.randint(10000) for i in range(10)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "stylegan2_repo_path/generate.py" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.7 --walk-type="{walk_types[1]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20}

5419,7241,4163,4072,3035,1258,6685,9753,4483,5436
Base seeds: [5419, 7241, 4163, 4072, 3035, 1258, 6685, 9753, 4483, 5436]
Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training_jap/00000-tfr_jap-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000088.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
slerp
Generating image for step 0/207 ...
Generating image for step 1/207 ...
Generating image for step 2/207 ...
Generating image for step 3/207 ...
Generating image for step 4/207 ...
Generating image for step 5/207 ...
Generating image for step 6/207 ...
Generating image for step 7/207 ...
Generating image for step 8/207 ...
Generating image for step 9/207 ...
Generating image for step 10/207 ...
Generating image for step 11/207 ...
Generating image for step 12/207 ...
Generating image for step 13/207 ...
Generating image for step 14/207 ...
Generating image for step 15/207 ...
Generati